In [1]:
import os
import gymnasium as gym
from algorithm.sabre_env import SabreSwapEnv

import qiskit.qasm2
from qiskit_ibm_runtime.fake_provider import FakeAlmadenV2
from qiskit.transpiler import CouplingMap

In [ ]:
# Set up the environment
provider = FakeAlmadenV2()
coupling_map = CouplingMap(provider.configuration().coupling_map)
data_path = '../data'
file_list = os.listdir(data_path)
file_list = [f for f in file_list if f.endswith('.qasm')]
paper_file_list = [
    '4mod5-v1_22.qasm',
    'mod5mils_65.qasm',
    'alu-v0_27.qasm',
    'decod24-v2_43.qasm',
    '4gt13_92.qasm',
    'ising_model_10.qasm',
    'ising_model_13.qasm',
    'ising_model_16.qasm',
    'qft_10.qasm',
    'qft_13.qasm',
    'qft_16.qasm',
    'qft_20.qasm',
    'rd84_142.qasm',
    'adr4_197.qasm',
    'radd_250.qasm',
    'z4_268.qasm',
    'sym6_145.qasm',
    'misex1_241.qasm',
    'rd73_252.qasm',
    'cycle10_2_110.qasm',
    'square_root_7.qasm',
    'sqn_258.qasm',
    'rd84_253.qasm',
    'co14_215.qasm',
    'sym9_193.qasm',
    '9symml_195.qasm',
]
paper_file_list = [file for file in file_list if file in paper_file_list]
file_list = [f for i, f in enumerate(file_list) if i % 5 == 0]
circuits = [qiskit.qasm2.load(os.path.join(data_path, f)) for f in file_list]
gym.register(
    id="SaberSwap-v0", entry_point="algorithm.sabre_env:SabreSwapEnv")
env = gym.make(
    id="SaberSwap-v0", circuits = circuits, coupling_map=coupling_map)

In [ ]:
# The agent model for the environment input, candidate: list[tuple[int, int]], sabre_dag: graph, current_layout: list[int], distance_matrix: list[list[int]]
# action -> tuple[int, int]

# https://github.com/benedekrozemberczki/graph2vec
# 그래프를 분석하는 것의 위의 embedding을 사용하면될 것 같다.
# 일단 저것을 사용하고 만약에 성능이 나오지 않는다면, GCN의 접근방식으로 그냥 모델을 바꾸어보자.